<a href="https://colab.research.google.com/github/AngelFelipeMP/Machine-Learning-Tweets-Classification/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### add NLP dependences
import sys
sys.path.append('drive/MyDrive/Code/DETOXIS/Machine-Learning-Tweets-Classification/Statistical_Models/')
from utils import process_tweet
import nltk     
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer

### add ML dependences
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB, ComplementNB, CategoricalNB
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix,  classification_report
from numpy import mean

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Functions

In [ ]:
#criate function process twittes
def preprocess_data(data):

  ### Preprocess data and so add it to a new column 'clean_comments_list'
  data["clean_comments_list"] = data["comment"].apply(lambda x : process_tweet(x))

  ### Removed preprocess data from a list and add it to a new collum called 'clean_comments_string'
  data["clean_comments_string"] = data["clean_comments_list"].apply(lambda x: ' '.join(x))

  return data

In [ ]:
#### criate function to Vectorize and extract features from twittes
def feature_extraction(data, pipeline, labels):

  for type_vec, value in pipeline.items():
    for ngr in value.keys():

      if type_vec == 'tfidf':
        pipeline[type_vec][ngr]['vect'] = TfidfVectorizer(ngram_range=ngr)
        pipeline[type_vec][ngr]['X'] = pipeline[type_vec][ngr]['vect'].fit_transform(data.clean_comments_string)
        pipeline[type_vec][ngr]['skf'] = StratifiedKFold(n_splits=10)
        pipeline[type_vec][ngr]['skf'].get_n_splits(pipeline[type_vec][ngr]['X'], labels)

      else:
        pipeline[type_vec][ngr]['vect'] = CountVectorizer(ngram_range=ngr)
        pipeline[type_vec][ngr]['X'] = pipeline[type_vec][ngr]['vect'].fit_transform(data.clean_comments_string)
        pipeline[type_vec][ngr]['skf'] = StratifiedKFold(n_splits=10)
        pipeline[type_vec][ngr]['skf'].get_n_splits(pipeline[type_vec][ngr]['X'], labels)

  return pipeline

In [ ]:
#### Handcraft metric for multiclass evaluation
def cem_metric(conf_metrix):
  cem_metrix = np.zeros(conf_metrix.shape)

  for column in range(conf_metrix.shape[1]):
    for row in range(conf_metrix.shape[0]):
    
      if row == column :
        cem_metrix[row,column] = (conf_metrix.sum(axis=0)[column]/2)/conf_metrix.sum()
                                          
      elif row < column:
        cem_metrix[row,column] = (conf_metrix.sum(axis=0)[column]/2 + conf_metrix.sum(axis=0)[row:column].sum())/conf_metrix.sum()

      elif row > column:
        cem_metrix[row,column] = (conf_metrix.sum(axis=0)[column]/2 + conf_metrix.sum(axis=0)[column+1:row+1].sum())/conf_metrix.sum()

  cem_metrix= - np.log2( np.where(cem_metrix !=0, cem_metrix, cem_metrix+0000000.1 ))

  return np.sum(cem_metrix * conf_metrix.T) / np.sum( np.diag(cem_metrix) * conf_metrix.sum(axis=0))

In [ ]:
#### criate function to train the models
def train_models(models, results, labels, pipe_dict, average_recall='binary', average_precision='binary', multiclass='False'):

  ### 10-Folds cross-validation
  for clf_name, clf in models.items():
    for nam_feature, values in pipe_dict.items():
      for n_gram, skf_tools in values.items():

      ## StratifiedKFold
        for train_index, test_index in skf_tools['skf'].split(skf_tools['X'], labels):

          X_train, X_test = skf_tools['X'][train_index], skf_tools['X'][test_index]
          y_train, y_test = labels[train_index], labels[test_index]

          if clf_name == 'gauNB_c':
            X_train = X_train.todense()
            X_test = X_test.todense()

          clf.fit(X_train, y_train)
          y_pred = clf.predict(X_test)

          results[clf_name][nam_feature][n_gram]['accuracy'].append(accuracy_score(y_test, y_pred))
          results[clf_name][nam_feature][n_gram]['recall'].append(recall_score(y_test, y_pred, average=average_recall))
          results[clf_name][nam_feature][n_gram]['precision'].append(precision_score(y_test, y_pred, average=average_precision))
          
          if multiclass == 'True':
          # if 'cem' in results[clf_name][nam_feature][n_gram]:
            results[clf_name][nam_feature][n_gram]['f1_macro'].append(f1_score(y_test, y_pred, average='macro'))
            results[clf_name][nam_feature][n_gram]['f1_weighted'].append(f1_score(y_test, y_pred, average='weighted'))
            results[clf_name][nam_feature][n_gram]['cem'].append(cem_metric(confusion_matrix(y_test, y_pred)))
          else:
            results[clf_name][nam_feature][n_gram]['f1'].append(f1_score(y_test, y_pred))

  return results

In [ ]:
### Create a fuction to average the models results
def average_results(average_dict, results_):

  ### Average model results
  for model_name, value in results_.items():
    for feature_vector, ngra_metric in value.items():
      for ngram, metrics_results in ngra_metric.items():
        for metric, values_results in metrics_results.items():

          average_dict[model_name][feature_vector][ngram][metric] = mean(values_results)

  return average_dict

In [ ]:
### create dataframe for our results
def create_Data_Frame(all_resultas):

  ### Criate a pandas da Frame with all results
  df_results = pd.DataFrame.from_dict({(i,j,k): all_resultas[i][j][k]
                            for i in all_resultas.keys()
                            for j in all_resultas[i].keys()
                            for k in all_resultas[i][j].keys()},
                        orient='index')
  return df_results

#Load data and Preprocess

In [ ]:
### Load data
data = pd.read_csv("drive/MyDrive/Code/DETOXIS/Data/train.csv")

In [ ]:
### check data/DataFrame
data.head()

,topic,thread_id,comment_id,reply_to,comment_level,comment,argumentation,constructiveness,positive_stance,negative_stance,target_person,target_group,stereotype,sarcasm,mockery,insult,improper_language,aggressiveness,intolerance,toxicity,toxicity_level
0,CR,0_000,0_002,0_002,1,Pensó: Zumo para restar.,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1
1,CR,0_001,0_003,0_003,1,Como les gusta el afeitado en seco a esta gente.,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1
2,CR,0_002,0_004,0_004,1,"asi me gusta, que se maten entre ellos y en al...",0,0,0,0,0,1,0,0,0,0,0,1,1,1,2
3,CR,0_003,0_005,0_005,1,"Loss mas valientes, los que mejor cortan nuest...",0,0,0,0,1,1,0,1,1,0,0,0,0,1,1
4,CR,0_004,0_006,0_006,1,Costumbres...,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1


In [ ]:
# Classes task1 and task 2
print('proportion classes task 1:\n',data['toxicity'].value_counts(),'\n')
print('proportion classes task 2:\n', data['toxicity_level'].value_counts())

proportion classes task 1:
 0    2316
1    1147
Name: toxicity, dtype: int64 

proportion classes task 2:
 0    2317
1     808
2     269
3      69
Name: toxicity_level, dtype: int64


In [ ]:
### create DataFrame with relevante information
data_train = data.loc[:,['comment','toxicity','toxicity_level']]

### check data/DataFrame
data_train.head()

,comment,toxicity,toxicity_level
0,Pensó: Zumo para restar.,1,1
1,Como les gusta el afeitado en seco a esta gente.,1,1
2,"asi me gusta, que se maten entre ellos y en al...",1,2
3,"Loss mas valientes, los que mejor cortan nuest...",1,1
4,Costumbres...,1,1


In [ ]:
# run fuction preprocess_data()
data_train = preprocess_data(data=data_train)

### check data/DataFrame
data_train.tail()

,comment,toxicity,toxicity_level,clean_comments_list,clean_comments_string
3458,Ya decía yo que veía menos moros,1,1,"[dec, vei, men, mor]",dec vei men mor
3459,+1. Como lo sabes...,0,0,"[numtag, sab, ...]",numtag sab ...
3460,"Seguirán cobrando paguitas en Marruecos,expoli...",1,1,"[segu, cobr, paguit, marruec, expoli, contribu...",segu cobr paguit marruec expoli contribuyent e...
3461,"pobres, se arriesgan en pateras porque huyen d...",1,1,"[pobr, arriesg, pater, huy, guerr, ..., ohhh, ...",pobr arriesg pater huy guerr ... ohhh whait ...
3462,"Yo me quiero escapar también, dan paguita al l...",1,1,"[quier, escap, dan, paguit, lleg]",quier escap dan paguit lleg


#Global dependeces

In [ ]:
### List of vectorization
vec_l = ['tfidf','occur']

### Feature extraction
ngrams = [(1,1), (1,2), (1,3)]

### Criate dictinaril to save import variables
pipeline_dict_task1 = { k:{ ngr: {'vect':0 , 'X':0, 'skf':0 } for ngr in ngrams } for k in vec_l }

### Label task1
y_task1 = data.toxicity

In [ ]:
### NB Criate classifier
multinomial = MultinomialNB()
bernoulli = BernoulliNB()
complement = ComplementNB()
gaussian = GaussianNB()

### NB classifiers
clfs = {'mulNB_c':multinomial, 'berNB_c':bernoulli, 'gauNB_c':gaussian, 'comNB_c':complement}

# Task 1

In [ ]:
### Evalute matrics
metrics_task1 = ['accuracy', 'f1', 'recall', 'precision']

### Criate dictinaril results
results_dict_task1 = { k:{ vec:{ ng:{m:[] for m in metrics_task1} for ng in ngrams} for vec in vec_l } for k,v in clfs.items() }

### Criate dictinaril results
mean_results_task1 = { k:{ vec:{ ng:{m:0 for m in metrics_task1} for ng in ngrams} for vec in vec_l } for k,v in clfs.items() }

In [ ]:
#### Execute function 'feature_extraction' to Vectorize and extract features from twittes
pipeline_dict_task1 = feature_extraction(data=data_train, pipeline=pipeline_dict_task1, labels=y_task1)

In [ ]:
### Apply function "train_models" and the results
results_dict_task1 = train_models(models=clfs, results=results_dict_task1, labels=y_task1, pipe_dict=pipeline_dict_task1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
### Apply function "average_results" to average the results whithin the 10 foldes
mean_results_task1 = average_results(average_dict=mean_results_task1, results_=results_dict_task1)

In [ ]:
### Apply function "reate_Data_Frame" 
df_results_task1 = create_Data_Frame(all_resultas=mean_results_task1)

In [ ]:
### save results to a CSV file
file_name = 'NB_results_task1'
df_results_task1.to_csv(r'drive/MyDrive/Code/DETOXIS/Machine-Learning-Tweets-Classification/Statistical_Models/Results/' + file_name + '.csv')

In [ ]:
### Results task 1
df_results_task1

accuracy        f1    recall  precision
mulNB_c tfidf (1, 1)  0.693293  0.170334  0.106217   0.728195
              (1, 2)  0.687809  0.099490  0.060885   0.716667
              (1, 3)  0.684348  0.080508  0.048703   0.550000
        occur (1, 1)  0.668473  0.486837  0.482113   0.495998
              (1, 2)  0.647978  0.523242  0.586781   0.473583
              (1, 3)  0.579550  0.535522  0.728894   0.423795
berNB_c tfidf (1, 1)  0.667373  0.334405  0.257353   0.497881
              (1, 2)  0.652350  0.191039  0.127353   0.429666
              (1, 3)  0.652927  0.176514  0.116003   0.416784
        occur (1, 1)  0.667373  0.334405  0.257353   0.497881
              (1, 2)  0.652350  0.191039  0.127353   0.429666
              (1, 3)  0.652927  0.176514  0.116003   0.416784
gauNB_c tfidf (1, 1)  0.472982  0.419167  0.583143   0.329405
              (1, 2)  0.528702  0.391791  0.473280   0.338637
              (1, 3)  0.530724  0.396140  0.479390   0.341846
        occur (1, 1)  0.467498  0.428201  0.610214   0.331683
              (1, 2)  0.522348  0.406760  0.509039   0.342470
              (1, 3)  0.524948  0.408367  0.509916   0.344243
comNB_c tfidf (1, 1)  0.660362  0.408290  0.379977   0.463310
              (1, 2)  0.678549  0.325519  0.264828   0.484494
              (1, 3)  0.683464  0.337801  0.272708   0.507133
        occur (1, 1)  0.623427  0.516491  0.615561   0.447195
              (1, 2)  0.592833  0.521595  0.674851   0.426254
              (1, 3)  0.521516  0.525603  0.800404   0.391500

#Task 2

In [ ]:
### check data/DataFrame
data_train.tail()

,comment,toxicity,toxicity_level,clean_comments_list,clean_comments_string
3458,Ya decía yo que veía menos moros,1,1,"[dec, vei, men, mor]",dec vei men mor
3459,+1. Como lo sabes...,0,0,"[numtag, sab, ...]",numtag sab ...
3460,"Seguirán cobrando paguitas en Marruecos,expoli...",1,1,"[segu, cobr, paguit, marruec, expoli, contribu...",segu cobr paguit marruec expoli contribuyent e...
3461,"pobres, se arriesgan en pateras porque huyen d...",1,1,"[pobr, arriesg, pater, huy, guerr, ..., ohhh, ...",pobr arriesg pater huy guerr ... ohhh whait ...
3462,"Yo me quiero escapar también, dan paguita al l...",1,1,"[quier, escap, dan, paguit, lleg]",quier escap dan paguit lleg


In [ ]:
### Criate dictinaril to save import variables
pipeline_dict_task2 = { k:{ ngr: {'vect':0 , 'X':0, 'skf':0 } for ngr in ngrams } for k in vec_l }

metrics_task2 = ['accuracy', 'f1_macro', 'f1_weighted', 'recall', 'precision', 'cem']

### Label task1
y_task2 = data.toxicity_level

In [ ]:
### Criate dictinaril results
results_dict_task2 = { k:{ vec:{ ng:{m:[] for m in metrics_task2} for ng in ngrams} for vec in vec_l } for k,v in clfs.items() }

### Criate dictinaril results
mean_results_task2 = { k:{ vec:{ ng:{m:0 for m in metrics_task2} for ng in ngrams} for vec in vec_l } for k,v in clfs.items() }

In [ ]:
#### Execute function 'feature_extraction' to Vectorize and extract features from twittes
pipeline_dict_task2 = feature_extraction(data=data_train, pipeline=pipeline_dict_task2, labels=y_task2)

In [ ]:
### Apply function "train_models" and get the results
results_dict_task2 = train_models(models=clfs, results=results_dict_task2, labels=y_task2, pipe_dict=pipeline_dict_task2, average_recall='micro', average_precision='micro',  multiclass='True')

In [ ]:
### Apply function "average_results" to average the results whithin the 10 foldes
mean_results_task2 = average_results(average_dict=mean_results_task2, results_=results_dict_task2)

In [ ]:
### Apply function "reate_Data_Frame" 
df_results_task2 = create_Data_Frame(all_resultas=mean_results_task2)

In [ ]:
### save results to a CSV file
file_name = 'NB_results_task2'
df_results_task2.to_csv(r'drive/MyDrive/Code/DETOXIS/Machine-Learning-Tweets-Classification/Statistical_Models/Results/' + file_name + '.csv')

In [ ]:
### Results task 1
df_results_task2

accuracy  f1_macro  ...  precision       cem
mulNB_c tfidf (1, 1)  0.674255  0.216046  ...   0.674255  0.680835
              (1, 2)  0.676854  0.216134  ...   0.676854  0.688164
              (1, 3)  0.676567  0.215761  ...   0.676567  0.688062
        occur (1, 1)  0.615069  0.273556  ...   0.615069  0.638369
              (1, 2)  0.606120  0.274716  ...   0.606120  0.647322
              (1, 3)  0.513725  0.265658  ...   0.513725  0.613320
berNB_c tfidf (1, 1)  0.622029  0.247250  ...   0.622029  0.621018
              (1, 2)  0.639636  0.221152  ...   0.639636  0.628622
              (1, 3)  0.639632  0.217458  ...   0.639632  0.626826
        occur (1, 1)  0.622029  0.247250  ...   0.622029  0.621018
              (1, 2)  0.639636  0.221152  ...   0.639636  0.628622
              (1, 3)  0.639632  0.217458  ...   0.639632  0.626826
gauNB_c tfidf (1, 1)  0.403125  0.231100  ...   0.403125  0.507530
              (1, 2)  0.492340  0.252957  ...   0.492340  0.537560
              (1, 3)  0.491477  0.253198  ...   0.491477  0.538570
        occur (1, 1)  0.399953  0.233312  ...   0.399953  0.508068
              (1, 2)  0.483389  0.253362  ...   0.483389  0.536091
              (1, 3)  0.483391  0.253848  ...   0.483391  0.536977
comNB_c tfidf (1, 1)  0.591080  0.274640  ...   0.591080  0.594795
              (1, 2)  0.648253  0.274885  ...   0.648253  0.634211
              (1, 3)  0.649697  0.279955  ...   0.649697  0.637679
        occur (1, 1)  0.493193  0.291643  ...   0.493193  0.575113
              (1, 2)  0.364696  0.250859  ...   0.364696  0.545922
              (1, 3)  0.200965  0.174927  ...   0.200965  0.492271

[24 rows x 6 columns]